In [ ]:
# Emotion Detection Preprocessing Notebook
# ResNet-18 Architecture for 48x48 Grayscale Images

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import make_grid
from collections import Counter
import json
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#CONFIG

CONFIG = {
    # Directory paths
    'dataset_root': '../../data/raw/fer2013/train',
    'train_dir': '../../data/raw/fer2013/train',
    'test_dir': '../../data/raw/fer2013/test',
    'output_dir': '../../data/processed_data',
    'model_save_dir': '../../models',
    
    # Dataset parameters
    'image_size': (48, 48),
    'batch_size': 32,
    'validation_split': 0.2,
    'test_split': 0.1,
    
    # Training parameters
    'num_epochs': 50,
    'learning_rate': 0.001,
    'weight_decay': 1e-4,
    'patience': 10,
    
    # Emotion labels
    'emotion_labels': ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
}

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Create output directories
for dir_path in [CONFIG['output_dir'], CONFIG['model_save_dir']]:
    os.makedirs(dir_path, exist_ok=True)

Environment Setup Complete!
PyTorch Version: 2.1.0+cu121
CUDA Available: False


In [ ]:
# =============================================================================
# EFFICIENT DATASET CLASS (No EDA)
# =============================================================================

class EmotionDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.data = []
        self.labels = []
        self.label_encoder = LabelEncoder()
        
        # Load images from directory structure
        for emotion in CONFIG['emotion_labels']:
            emotion_dir = os.path.join(data_dir, emotion)
            if os.path.exists(emotion_dir):
                for img_file in os.listdir(emotion_dir):
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_path = os.path.join(emotion_dir, img_file)
                        self.data.append(img_path)
                        self.labels.append(emotion)
        
        # Encode labels
        self.labels = self.label_encoder.fit_transform(self.labels)
        print(f"Loaded {len(self.data)} images across {len(self.label_encoder.classes_)} classes")
        
        # Print class distribution
        self.print_class_distribution()
    
    def print_class_distribution(self):
        """Print and visualize class distribution"""
        class_counts = Counter(self.labels)
        print("\nClass Distribution:")
        for i, class_name in enumerate(self.label_encoder.classes_):
            count = class_counts[i]
            percentage = (count / len(self.labels)) * 100
            print(f"{class_name}: {count} images ({percentage:.1f}%)")
    
    def get_class_weights(self):
        """Calculate class weights for handling imbalanced dataset"""
        class_weights = compute_class_weight(
            'balanced',
            classes=np.unique(self.labels),
            y=self.labels
        )
        return torch.FloatTensor(class_weights)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data[idx]
        label = self.labels[idx]
        
        # Load image
        image = Image.open(img_path)
        
        # Convert to grayscale if needed
        if image.mode != 'L':
            image = image.convert('L')
        
        # Resize if needed
        if image.size != CONFIG['image_size']:
            image = image.resize(CONFIG['image_size'])
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [ ]:
# =============================================================================
# OPTIMIZED TRANSFORMS (No Resize Needed)
# =============================================================================
class TransformedDataset(Dataset):
    """Wrapper to apply specific transforms to dataset subsets"""
    def __init__(self, dataset, indices, transform=None):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        actual_idx = self.indices[idx]
        img_path = self.dataset.data[actual_idx]
        label = self.dataset.labels[actual_idx]
        
        # Load image
        image = Image.open(img_path)
        if image.mode != 'L':
            image = image.convert('L')
        if image.size != CONFIG['image_size']:
            image = image.resize(CONFIG['image_size'])
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
def calculate_dataset_statistics(dataset):
    """Calculate mean and std of the dataset"""
    print("Calculating dataset statistics...")
    
    # Create a simple transform to convert to tensor
    temp_transform = transforms.Compose([transforms.ToTensor()])
    
    # Calculate statistics
    pixel_sum = 0
    pixel_squared_sum = 0
    num_pixels = 0
    
    for i in range(len(dataset)):
        img_path = dataset.data[i]
        image = Image.open(img_path)
        if image.mode != 'L':
            image = image.convert('L')
        if image.size != CONFIG['image_size']:
            image = image.resize(CONFIG['image_size'])
        
        # Convert to tensor
        image_tensor = temp_transform(image)
        
        # Calculate statistics
        pixel_sum += image_tensor.sum()
        pixel_squared_sum += (image_tensor ** 2).sum()
        num_pixels += image_tensor.numel()
    
    mean = pixel_sum / num_pixels
    std = torch.sqrt(pixel_squared_sum / num_pixels - mean ** 2)
    
    print(f"Dataset statistics - Mean: {mean:.4f}, Std: {std:.4f}")
    return mean.item(), std.item()

In [ ]:
def get_transforms(mean=None, std=None):
    """Get transforms with proper normalization"""
    # Use calculated statistics or default values
    if mean is None or std is None:
        mean, std = 0.485, 0.229  # Default ImageNet values
    
    # Training transforms with augmentation
    train_transforms = transforms.Compose([
        transforms.RandomRotation(15),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean], std=[std])
    ])
    
    # Validation/Test transforms
    val_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean], std=[std])
    ])
    
    return train_transforms, val_transforms

In [ ]:
class EmotionResNet(nn.Module):
    def __init__(self, num_classes=7, pretrained=True, dropout_rate=0.5):
        super(EmotionResNet, self).__init__()
        
        # Load pretrained ResNet-18
        self.resnet = models.resnet18(pretrained=pretrained)
        
        # Modify first layer for grayscale input
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        # Modify final layer for emotion classification
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout_rate * 0.6),
            nn.Linear(512, num_classes)
        )
        
        # Initialize the new conv1 layer
        nn.init.kaiming_normal_(self.resnet.conv1.weight, mode='fan_out', nonlinearity='relu')
    
    def forward(self, x):
        return self.resnet(x)

In [ ]:
def visualize_sample_images(dataset, num_samples=16):
    """Visualize sample images from each class"""
    fig, axes = plt.subplots(2, 8, figsize=(20, 6))
    axes = axes.flatten()
    
    # Get samples from each class
    class_samples = {}
    for i, label in enumerate(dataset.labels):
        if label not in class_samples:
            class_samples[label] = []
        if len(class_samples[label]) < 2:
            class_samples[label].append(i)
    
    # Display samples
    idx = 0
    for class_idx, class_name in enumerate(dataset.label_encoder.classes_):
        if class_idx in class_samples:
            for sample_idx in class_samples[class_idx]:
                if idx < 16:
                    img_path = dataset.data[sample_idx]
                    image = Image.open(img_path)
                    if image.mode != 'L':
                        image = image.convert('L')
                    
                    axes[idx].imshow(image, cmap='gray')
                    axes[idx].set_title(f'{class_name}', fontsize=10)
                    axes[idx].axis('off')
                    idx += 1
    
    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['output_dir'], 'sample_images.png'), dpi=300, bbox_inches='tight')
    plt.show()

def plot_class_distribution(dataset):
    """Plot class distribution"""
    class_counts = Counter(dataset.labels)
    classes = [dataset.label_encoder.classes_[i] for i in range(len(dataset.label_encoder.classes_))]
    counts = [class_counts[i] for i in range(len(dataset.label_encoder.classes_))]
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(classes, counts, color='steelblue', alpha=0.7)
    plt.title('Class Distribution in Dataset', fontsize=14, fontweight='bold')
    plt.xlabel('Emotion Classes', fontsize=12)
    plt.ylabel('Number of Images', fontsize=12)
    plt.xticks(rotation=45)
    
    # Add value labels on bars
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
                str(count), ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['output_dir'], 'class_distribution.png'), dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
def create_data_loaders():
    """Create train/validation/test data loaders with proper transforms"""
    
    print("Creating data loaders...")
    
    # Load dataset
    full_dataset = EmotionDataset(CONFIG['dataset_root'])
    
    # Calculate dataset statistics
    mean, std = calculate_dataset_statistics(full_dataset)
    
    # Create data splits
    train_idx, temp_idx = train_test_split(
        range(len(full_dataset)), 
        test_size=CONFIG['validation_split'] + CONFIG['test_split'],
        stratify=full_dataset.labels, 
        random_state=42
    )
    
    val_idx, test_idx = train_test_split(
        temp_idx,
        test_size=CONFIG['test_split'] / (CONFIG['validation_split'] + CONFIG['test_split']),
        stratify=[full_dataset.labels[i] for i in temp_idx], 
        random_state=42
    )
    
    print(f"Data splits - Train: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")
    
    # Get transforms with calculated statistics
    train_transforms, val_transforms = get_transforms(mean, std)
    
    # Create transformed datasets
    train_dataset = TransformedDataset(full_dataset, train_idx, train_transforms)
    val_dataset = TransformedDataset(full_dataset, val_idx, val_transforms)
    test_dataset = TransformedDataset(full_dataset, test_idx, val_transforms)
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=CONFIG['batch_size'], 
        shuffle=True, 
        num_workers=4, 
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset, 
        batch_size=CONFIG['batch_size'], 
        shuffle=False, 
        num_workers=4, 
        pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset, 
        batch_size=CONFIG['batch_size'], 
        shuffle=False, 
        num_workers=4, 
        pin_memory=True
    )
    
    return train_loader, val_loader, test_loader, full_dataset, mean, std

def setup_model_and_training():
    """Setup model and training components"""
    
    print("Setting up model and training components...")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Initialize model
    model = EmotionResNet(num_classes=len(CONFIG['emotion_labels']))
    model = model.to(device)
    
    # Model summary
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    
    return model, device

def save_preprocessing_info(dataset, train_size, val_size, test_size, mean, std, class_weights):
    """Save preprocessing information"""
    
    preprocessing_info = {
        'dataset_size': len(dataset),
        'train_size': train_size,
        'val_size': val_size,
        'test_size': test_size,
        'num_classes': len(CONFIG['emotion_labels']),
        'emotion_labels': CONFIG['emotion_labels'],
        'label_encoder_classes': dataset.label_encoder.classes_.tolist(),
        'image_size': CONFIG['image_size'],
        'batch_size': CONFIG['batch_size'],
        'dataset_statistics': {
            'mean': mean,
            'std': std
        },
        'class_weights': class_weights.tolist(),
        'config': CONFIG
    }
    
    with open(os.path.join(CONFIG['output_dir'], 'preprocessing_info.json'), 'w') as f:
        json.dump(preprocessing_info, f, indent=2)
    
    print(f"Preprocessing info saved to: {os.path.join(CONFIG['output_dir'], 'preprocessing_info.json')}")
    return preprocessing_info

EMOTION DETECTION PREPROCESSING
Creating data loaders...
Loaded 0 images across 0 classes


ValueError: With n_samples=0, test_size=0.30000000000000004 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
def main():
    """Main preprocessing pipeline execution"""
    
    print("=" * 60)
    print("EMOTION DETECTION PREPROCESSING - ENHANCED VERSION")
    print("=" * 60)
    
    # Create data loaders
    train_loader, val_loader, test_loader, dataset, mean, std = create_data_loaders()
    
    # Setup model
    model, device = setup_model_and_training()
    
    # Get class weights for handling imbalanced dataset
    class_weights = dataset.get_class_weights()
    print(f"Class weights: {class_weights}")
    
    # Visualize data
    print("\nGenerating visualizations...")
    plot_class_distribution(dataset)
    visualize_sample_images(dataset)
    
    # Save preprocessing information
    preprocessing_info = save_preprocessing_info(
        dataset, 
        len(train_loader.dataset), 
        len(val_loader.dataset), 
        len(test_loader.dataset),
        mean, 
        std,
        class_weights
    )
    
    print("\n" + "=" * 60)
    print("PREPROCESSING COMPLETED SUCCESSFULLY!")
    print("=" * 60)
    print(f"Dataset size: {len(dataset)} images")
    print(f"Classes: {len(CONFIG['emotion_labels'])}")
    print(f"Dataset statistics: Mean={mean:.4f}, Std={std:.4f}")
    print("Ready for training!")
    
    return {
        'train_loader': train_loader,
        'val_loader': val_loader,
        'test_loader': test_loader,
        'model': model,
        'device': device,
        'class_weights': class_weights,
        'dataset_stats': {'mean': mean, 'std': std},
        'preprocessing_info': preprocessing_info
    }


In [ ]:
if __name__ == "__main__":
    results = main()
    
    # Now you can use the results for training:
    # train_loader = results['train_loader']
    # val_loader = results['val_loader']
    # test_loader = results['test_loader']
    # model = results['model']
    # device = results['device']
    # class_weights = results['class_weights']